In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Takoradi,GH,2022-07-18 01:08:39,4.8845,-1.7554,74.01,82,75,8.99
1,1,Zhaodong,CN,2022-07-18 01:08:39,46.0833,125.9833,73.89,73,100,5.21
2,2,Esperance,AU,2022-07-18 01:08:40,-33.8667,121.9000,53.87,89,100,20.13
3,3,Bredasdorp,ZA,2022-07-18 01:08:40,-34.5322,20.0403,52.21,88,1,4.16
4,4,Atuona,PF,2022-07-18 01:08:40,-9.8000,-139.0333,78.22,77,22,19.35


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [9]:
#Get the latitude and longitude
locations = city_data_df[['Lat', 'Lng']]

#Assign the figure variable
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#Assign heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp,0) for temp in max_temp], 
                                 dissipating=False, max_intensity=300, point_radius=4)

#Add the heatmap layer
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
locations = city_data_df[['Lat', 'Lng']]
humidity = city_data_df['Humidity']
fig = gmaps.figure(center=(30.0, 31), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
locations = city_data_df[['Lat', 'Lng']]
clouds = city_data_df['Cloudiness']
fig = gmaps.figure(center=(30.0, 31), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, 
                                 dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [13]:
#Ask the customer to add a minimum and maximum temperature value
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip? '))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [14]:
#Filter the data set to find the cities that fit the crieteria
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & \
                                       (city_data_df['Max Temp'] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
4,4,Atuona,PF,2022-07-18 01:08:40,-9.8000,-139.0333,78.22,77,22,19.35
9,9,Broome,US,2022-07-18 01:08:41,42.2506,-75.8330,75.97,74,20,5.75
17,17,Shingu,JP,2022-07-18 01:08:44,33.7333,135.9833,85.48,59,98,4.16
19,19,Iberia,US,2022-07-18 01:08:45,29.9502,-91.7507,85.96,82,97,6.91
22,22,Vaini,TO,2022-07-18 01:07:51,-21.2000,-175.2000,78.96,89,40,9.22
24,24,Sirjan,IR,2022-07-18 01:08:46,29.4520,55.6814,76.32,12,1,4.68
25,25,Kudahuvadhoo,MV,2022-07-18 01:08:46,2.6708,72.8944,77.18,87,100,17.78
30,30,Buchanan,LR,2022-07-18 01:05:21,5.8808,-10.0467,75.99,84,100,11.43
32,32,Cayenne,GF,2022-07-18 01:06:33,4.9333,-52.3333,83.08,1,0,0.00
34,34,Thompson,CA,2022-07-18 01:06:45,55.7435,-97.8558,78.31,38,100,9.10


In [15]:
preferred_cities_df.count()

City_ID       199
City          199
Country       199
Date          199
Lat           199
Lng           199
Max Temp      199
Humidity      199
Cloudiness    199
Wind Speed    199
dtype: int64

In [16]:
#Create a data frame called hotel_df to store hotel names along with city, country, max temp, and coordinates
hotel_df = preferred_cities_df[['City', 'Country', 'Max Temp', 'Lat', 'Lng']].copy()
hotel_df['Hotel Name'] = ''
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Atuona,PF,78.22,-9.8000,-139.0333,
9,Broome,US,75.97,42.2506,-75.8330,
17,Shingu,JP,85.48,33.7333,135.9833,
19,Iberia,US,85.96,29.9502,-91.7507,
22,Vaini,TO,78.96,-21.2000,-175.2000,
24,Sirjan,IR,76.32,29.4520,55.6814,
25,Kudahuvadhoo,MV,77.18,2.6708,72.8944,
30,Buchanan,LR,75.99,5.8808,-10.0467,
32,Cayenne,GF,83.08,4.9333,-52.3333,
34,Thompson,CA,78.31,55.7435,-97.8558,


In [17]:
#Set parameters to search for a hotel
params = {
    'radius': 5000,
    'type': 'lodging',
    'key': g_key
}

#Iterate through the data frame
for index, row in hotel_df.iterrows():
    #Get the latitude and longitude
    lat = row['Lat']
    lng = row['Lng']
    
    #Add the latitude and longitude to location key for the params dictionary
    params['location'] = f"{lat},{lng}"
    
    #Use the search term: "lodging" and our latitude and longitude
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    #Make request and get the JSON data from the search
    hotels = requests.get(base_url, params=params).json()
    #Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping.')

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [18]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Atuona,PF,78.22,-9.8000,-139.0333,Villa Enata
9,Broome,US,75.97,42.2506,-75.8330,Chenango Valley State Park
17,Shingu,JP,85.48,33.7333,135.9833,Shingu UI Hotel
19,Iberia,US,85.96,29.9502,-91.7507,back to back rv park
22,Vaini,TO,78.96,-21.2000,-175.2000,Keleti Beach Resort
24,Sirjan,IR,76.32,29.4520,55.6814,MAPNA Building
25,Kudahuvadhoo,MV,77.18,2.6708,72.8944,Niyama Private Islands Maldives
30,Buchanan,LR,75.99,5.8808,-10.0467,Kamaneahn Hotel Inc
32,Cayenne,GF,83.08,4.9333,-52.3333,Hôtel Le Dronmi
34,Thompson,CA,78.31,55.7435,-97.8558,Burntwood Hotel


In [19]:
#Add a heatmap of temperature for vacation spots
locations = hotel_df[['Lat', 'Lng']]
max_temp = hotel_df['Max Temp']
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, 
                                 max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

#Store the data frame row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [21]:
#Add a heatmap of temperature for vacation spots and a pop-up marker for each city
locations = hotel_df[['Lat', 'Lng']]
max_temp = hotel_df['Max Temp']
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, 
                                 max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))